In [4]:
import pandas as pd
import requests
import io

# URLs oficiales (verificadas)
URL_DEMO = "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT"
URL_CBC = "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/CBC_J.XPT"

def download_and_read(url, name):
    print(f"⬇️ Descargando {name}...")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        
        # Verificar si la descarga es válida (no error 404 oculto)
        if response.status_code != 200:
            print(f"❌ Error HTTP {response.status_code} en {name}")
            return None
            
        # Verificar si bajamos un HTML de error en vez de datos binarios
        if b"<!DOCTYPE html>" in response.content[:100] or b"<html" in response.content[:100]:
            print(f"❌ Error: El servidor devolvió una página web, no el archivo de datos para {name}.")
            return None

        # CORRECCIÓN CRÍTICA: Especificar format='xport'
        return pd.read_sas(io.BytesIO(response.content), format='xport')

    except Exception as e:
        print(f"❌ Falló la carga de {name}: {e}")
        return None

def main():
    # 1. Intentar descarga automática
    df_demo = download_and_read(URL_DEMO, "Demografía")
    df_cbc = download_and_read(URL_CBC, "Laboratorio")
    
    # 2. PLAN DE EMERGENCIA (Carga manual si falla la descarga)
    if df_demo is None or df_cbc is None:
        print("\n" + "="*50)
        print("🚨 LA DESCARGA AUTOMÁTICA FALLÓ (Bloqueo de red/CDC)")
        print("⚡ ACCIÓN MANUAL REQUERIDA:")
        print("1. Descarga estos 2 archivos y guárdalos en ESTA misma carpeta:")
        print(f"   - {URL_DEMO}")
        print(f"   - {URL_CBC}")
        print("2. Una vez descargados, presiona ENTER para continuar...")
        print("="*50)
        input("Presiona ENTER cuando los archivos 'DEMO_J.XPT' y 'CBC_J.XPT' estén en la carpeta...")
        
        try:
            print("📂 Leyendo archivos locales...")
            df_demo = pd.read_sas('DEMO_J.XPT')
            df_cbc = pd.read_sas('CBC_J.XPT')
        except FileNotFoundError:
            print("❌ No encuentro los archivos. Asegúrate de que se llamen EXACTAMENTE 'DEMO_J.XPT' y 'CBC_J.XPT'")
            return

    # 3. Fusión y Limpieza (Si llegamos aquí, tenemos datos)
    print("✅ Datos cargados. Procesando...")
    df_merged = pd.merge(df_demo, df_cbc, on="SEQN", how="inner")
    
    column_mapping = {
        'SEQN': 'ID', 'RIAGENDR': 'Sex', 'RIDAGEYR': 'Age',
        'LBXWBCSI': 'WBC', 'LBXNEPCT': 'NEU_percent', 'LBXLYPCT': 'LYM_percent',
        'LBXRBCSI': 'RBC', 'LBXHGB': 'HGB', 'LBXHCT': 'HCT',
        'LBXMCV': 'MCV', 'LBXMCH': 'MCH', 'LBXPLTSI': 'PLT'
    }
    
    # Filtrar y Renombrar
    avail_cols = [c for c in column_mapping.keys() if c in df_merged.columns]
    df_clean = df_merged[avail_cols].rename(columns=column_mapping)
    
    # Vectorización
    if 'WBC' in df_clean.columns:
        if 'NEU_percent' in df_clean.columns:
            df_clean['NEU'] = df_clean['WBC'] * (df_clean['NEU_percent'] / 100)
        if 'LYM_percent' in df_clean.columns:
            df_clean['LYM'] = df_clean['WBC'] * (df_clean['LYM_percent'] / 100)
            
    # Sanity Drop
    targets = ['HGB', 'HCT', 'RBC', 'MCV', 'MCH', 'WBC', 'PLT']
    final_targets = [t for t in targets if t in df_clean.columns]
    df_clean = df_clean.dropna(subset=final_targets)
    
    print(f"🎉 ÉXITO TOTAL: Dataset de {df_clean.shape[0]} pacientes generado.")
    df_clean.to_csv("nhanes_raw_data.csv", index=False)

if __name__ == "__main__":
    main()

⬇️ Descargando Demografía...
❌ Error: El servidor devolvió una página web, no el archivo de datos para Demografía.
⬇️ Descargando Laboratorio...
❌ Error: El servidor devolvió una página web, no el archivo de datos para Laboratorio.

🚨 LA DESCARGA AUTOMÁTICA FALLÓ (Bloqueo de red/CDC)
⚡ ACCIÓN MANUAL REQUERIDA:
1. Descarga estos 2 archivos y guárdalos en ESTA misma carpeta:
   - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT
   - https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/CBC_J.XPT
2. Una vez descargados, presiona ENTER para continuar...
📂 Leyendo archivos locales...
✅ Datos cargados. Procesando...
🎉 ÉXITO TOTAL: Dataset de 7528 pacientes generado.
